In [1]:
import os
import re
import sys

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import LatentDirichletAllocation


import nltk
import spacy

In [2]:
from gensim import corpora, models, similarities, matutils

# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
df_news=[]
for i in range(1,4):
    df_news.append(pd.read_csv('articles'+str(i)+'.csv', index_col='id'))
df_news = pd.concat(df_news)

print(df_news.shape)
df_news.head()

(142570, 9)


,Unnamed: 0,title,publication,author,date,year,month,url,content
id,,,,,,,,,
17283,0,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
17284,1,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
17285,2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
17286,3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
17287,4,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [4]:
df_news = df_news.drop(['Unnamed: 0', 'url'], axis=1)
df_news.isnull().sum()

title              2
publication        0
author         15876
date            2641
year            2641
month           2641
content            0
dtype: int64

In [5]:
df_news = df_news.dropna()

In [6]:
x = df_news.publication.value_counts()
print(x[x>100].index[:])

Index(['Breitbart', 'New York Post', 'NPR', 'Washington Post', 'Reuters',
       'New York Times', 'Guardian', 'CNN', 'National Review', 'Atlantic',
       'Business Insider', 'Vox', 'Buzzfeed News', 'Fox News'],
      dtype='object')


In [7]:
x = df_news.author.value_counts()
print(x[x>100].index[:])
np.mean(len(df_news.title.iloc[:]))

Index(['Breitbart News', 'Pam Key', 'Associated Press', 'Charlie Spiering',
       'Jerome Hudson', 'John Hayward', 'Daniel Nussbaum', 'AWR Hawkins',
       'Ian Hanchett', 'Joel B. Pollak',
       ...
       'Brian Naylor', 'Daniella Diaz', 'Jamie Schram', 'Oliver Darcy',
       'Carleton English', 'Robinson Meyer', 'Ed Yong', 'Matt Zapotosky',
       'John Wagner', 'Cartel Chronicles'],
      dtype='object', length=216)


125010.0

In [8]:
X=df_news.title
y=df_news.publication
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42)

In [9]:
cv1 = CountVectorizer(ngram_range=(1,2), binary=False, stop_words='english', 
                      lowercase=True, strip_accents='unicode', max_df=0.5)

X_train_cv1 = cv1.fit_transform(X_train)
X_train_cv1_transpose = cv1.fit_transform(X_train).transpose()
X_test_cv1  = cv1.transform(X_test)

In [10]:
pd.DataFrame(X_train_cv1_transpose.toarray(), cv1.get_feature_names()).tail()

,0,1,2,3,4,5,6,7,8,9,...,37493,37494,37495,37496,37497,37498,37499,37500,37501,37502
zurich,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zurich islamic,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zurich mosque,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zverev,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zverev sets,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
X_train_cv1_transpose.shape

(205317, 37503)

In [12]:
corpus = matutils.Sparse2Corpus(X_train_cv1_transpose)

In [13]:
id2word = dict((v, k) for k, v in cv1.vocabulary_.items())

In [14]:
len(id2word)

205317

### LDA

In [15]:
lda = models.LdaModel(corpus=corpus, num_topics=5, id2word=id2word, passes=10)

2020-02-25 12:53:02,919 : INFO : using symmetric alpha at 0.2
2020-02-25 12:53:02,921 : INFO : using symmetric eta at 0.2
2020-02-25 12:53:02,945 : INFO : using serial LDA version on this node
2020-02-25 12:53:03,045 : INFO : running online (multi-pass) LDA training, 5 topics, 10 passes over the supplied corpus of 37503 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2020-02-25 12:53:03,056 : INFO : PROGRESS: pass 0, at document #2000/37503
2020-02-25 12:53:04,284 : INFO : merging changes from 2000 documents into a model of 37503 documents
2020-02-25 12:53:04,328 : INFO : topic #0 (0.200): 0.008*"breitbart" + 0.007*"trump" + 0.004*"new" + 0.002*"new york" + 0.002*"york" + 0.002*"times" + 0.001*"york times" + 0.001*"obama" + 0.001*"china" + 0.001*"world"
2020-02-25 12:53:04,331 : INFO : topic #1 (0.200): 0.015*"trump" + 0.007*"new" + 0.007*"breitbart" + 0.005*"york" + 0.005*"new york

2020-02-25 12:53:07,978 : INFO : topic #4 (0.200): 0.022*"new" + 0.017*"york" + 0.017*"new york" + 0.016*"times" + 0.016*"york times" + 0.014*"trump" + 0.007*"breitbart" + 0.003*"clinton" + 0.002*"obama" + 0.002*"hillary"
2020-02-25 12:53:07,980 : INFO : topic diff=0.235099, rho=0.408248
2020-02-25 12:53:07,997 : INFO : PROGRESS: pass 0, at document #14000/37503
2020-02-25 12:53:08,613 : INFO : merging changes from 2000 documents into a model of 37503 documents
2020-02-25 12:53:08,660 : INFO : topic #0 (0.200): 0.008*"breitbart" + 0.005*"trump" + 0.003*"obama" + 0.002*"gop" + 0.002*"world" + 0.002*"work" + 0.002*"didn" + 0.002*"health" + 0.002*"obamacare" + 0.002*"wants"
2020-02-25 12:53:08,666 : INFO : topic #1 (0.200): 0.029*"trump" + 0.017*"breitbart" + 0.008*"donald" + 0.008*"donald trump" + 0.008*"clinton" + 0.005*"hillary" + 0.004*"new" + 0.004*"obama" + 0.003*"hillary clinton" + 0.003*"says"
2020-02-25 12:53:08,671 : INFO : topic #2 (0.200): 0.005*"breitbart" + 0.005*"trump" + 0

2020-02-25 12:53:12,711 : INFO : topic #4 (0.200): 0.028*"new" + 0.021*"york" + 0.021*"new york" + 0.020*"times" + 0.019*"york times" + 0.011*"trump" + 0.004*"breitbart" + 0.002*"obama" + 0.002*"police" + 0.002*"years"
2020-02-25 12:53:12,712 : INFO : topic diff=0.222085, rho=0.288675
2020-02-25 12:53:12,729 : INFO : PROGRESS: pass 0, at document #26000/37503
2020-02-25 12:53:13,240 : INFO : merging changes from 2000 documents into a model of 37503 documents
2020-02-25 12:53:13,285 : INFO : topic #0 (0.200): 0.006*"breitbart" + 0.003*"trump" + 0.003*"health" + 0.002*"care" + 0.002*"obamacare" + 0.002*"china" + 0.002*"health care" + 0.002*"gop" + 0.002*"work" + 0.002*"like"
2020-02-25 12:53:13,290 : INFO : topic #1 (0.200): 0.033*"trump" + 0.022*"breitbart" + 0.009*"donald" + 0.009*"donald trump" + 0.009*"clinton" + 0.006*"hillary" + 0.004*"obama" + 0.004*"hillary clinton" + 0.004*"white" + 0.003*"house"
2020-02-25 12:53:13,294 : INFO : topic #2 (0.200): 0.004*"court" + 0.004*"breitbart

2020-02-25 12:53:16,385 : INFO : topic diff=0.209211, rho=0.235702
2020-02-25 12:53:17,030 : INFO : -12.855 per-word bound, 7407.6 perplexity estimate based on a held-out corpus of 1503 documents with 20235 words
2020-02-25 12:53:17,031 : INFO : PROGRESS: pass 0, at document #37503/37503
2020-02-25 12:53:17,398 : INFO : merging changes from 1503 documents into a model of 37503 documents
2020-02-25 12:53:17,439 : INFO : topic #0 (0.200): 0.005*"breitbart" + 0.003*"health" + 0.002*"care" + 0.002*"obamacare" + 0.002*"like" + 0.002*"trump" + 0.002*"work" + 0.002*"health care" + 0.002*"tax" + 0.002*"china"
2020-02-25 12:53:17,441 : INFO : topic #1 (0.200): 0.035*"trump" + 0.023*"breitbart" + 0.009*"donald" + 0.009*"clinton" + 0.009*"donald trump" + 0.006*"hillary" + 0.005*"obama" + 0.004*"hillary clinton" + 0.003*"white" + 0.003*"says"
2020-02-25 12:53:17,443 : INFO : topic #2 (0.200): 0.005*"court" + 0.003*"breitbart" + 0.003*"wall" + 0.002*"supreme" + 0.002*"supreme court" + 0.002*"says" 

2020-02-25 12:53:20,227 : INFO : topic diff=0.191099, rho=0.219521
2020-02-25 12:53:20,244 : INFO : PROGRESS: pass 1, at document #12000/37503
2020-02-25 12:53:20,864 : INFO : merging changes from 2000 documents into a model of 37503 documents
2020-02-25 12:53:20,920 : INFO : topic #0 (0.200): 0.005*"breitbart" + 0.003*"health" + 0.002*"obamacare" + 0.002*"work" + 0.002*"care" + 0.002*"world" + 0.002*"health care" + 0.002*"china" + 0.002*"like" + 0.002*"trump"
2020-02-25 12:53:20,923 : INFO : topic #1 (0.200): 0.035*"trump" + 0.023*"breitbart" + 0.009*"clinton" + 0.008*"donald" + 0.008*"donald trump" + 0.006*"hillary" + 0.004*"obama" + 0.004*"hillary clinton" + 0.003*"says" + 0.003*"white"
2020-02-25 12:53:20,927 : INFO : topic #2 (0.200): 0.004*"court" + 0.003*"breitbart" + 0.002*"high" + 0.002*"wall" + 0.002*"says" + 0.002*"supreme" + 0.002*"supreme court" + 0.002*"report" + 0.002*"street" + 0.002*"school"
2020-02-25 12:53:20,932 : INFO : topic #3 (0.200): 0.003*"man" + 0.003*"state"

...

2020-02-25 12:54:39,814 : INFO : topic #0 (0.200): 0.003*"breitbart" + 0.003*"health" + 0.002*"care" + 0.002*"like" + 0.002*"work" + 0.002*"obamacare" + 0.002*"health care" + 0.002*"left" + 0.001*"china" + 0.001*"world"
2020-02-25 12:54:39,817 : INFO : topic #1 (0.200): 0.036*"trump" + 0.024*"breitbart" + 0.009*"clinton" + 0.008*"donald" + 0.008*"donald trump" + 0.006*"hillary" + 0.005*"obama" + 0.003*"hillary clinton" + 0.003*"white" + 0.003*"says"
2020-02-25 12:54:39,822 : INFO : topic #2 (0.200): 0.004*"court" + 0.003*"wall" + 0.003*"breitbart" + 0.002*"street" + 0.002*"supreme" + 0.002*"supreme court" + 0.002*"high" + 0.002*"school" + 0.002*"pay" + 0.002*"wall street"
2020-02-25 12:54:39,827 : INFO : topic #3 (0.200): 0.003*"man" + 0.003*"year" + 0.003*"old" + 0.003*"state" + 0.003*"islamic" + 0.002*"police" + 0.002*"killed" + 0.002*"islamic state" + 0.002*"woman" + 0.002*"year old"
2020-02-25 12:54:39,830 : INFO : topic #4 (0.200): 0.031*"new" + 0.021*"york" + 0.021*"new york" + 0

2020-02-25 12:54:42,588 : INFO : topic #1 (0.200): 0.036*"trump" + 0.024*"breitbart" + 0.009*"clinton" + 0.009*"donald" + 0.008*"donald trump" + 0.006*"hillary" + 0.005*"obama" + 0.004*"hillary clinton" + 0.003*"house" + 0.003*"white"
2020-02-25 12:54:42,593 : INFO : topic #2 (0.200): 0.005*"court" + 0.003*"wall" + 0.003*"breitbart" + 0.002*"street" + 0.002*"supreme" + 0.002*"supreme court" + 0.002*"high" + 0.002*"says" + 0.002*"school" + 0.002*"wall street"
2020-02-25 12:54:42,596 : INFO : topic #3 (0.200): 0.004*"year" + 0.004*"man" + 0.003*"old" + 0.003*"state" + 0.003*"islamic" + 0.002*"year old" + 0.002*"islamic state" + 0.002*"police" + 0.002*"arrested" + 0.002*"killed"
2020-02-25 12:54:42,600 : INFO : topic #4 (0.200): 0.031*"new" + 0.021*"york" + 0.021*"new york" + 0.020*"times" + 0.019*"york times" + 0.008*"trump" + 0.002*"police" + 0.002*"ban" + 0.002*"years" + 0.001*"administration"
2020-02-25 12:54:42,602 : INFO : topic diff=0.168563, rho=0.189826
2020-02-25 12:54:42,617 : 

2020-02-25 12:54:45,709 : INFO : topic #1 (0.200): 0.035*"trump" + 0.024*"breitbart" + 0.009*"clinton" + 0.008*"donald" + 0.008*"donald trump" + 0.006*"hillary" + 0.005*"obama" + 0.004*"hillary clinton" + 0.003*"house" + 0.003*"white"
2020-02-25 12:54:45,713 : INFO : topic #2 (0.200): 0.004*"court" + 0.003*"breitbart" + 0.002*"wall" + 0.002*"street" + 0.002*"high" + 0.002*"supreme" + 0.002*"supreme court" + 0.002*"zika" + 0.002*"says" + 0.002*"report"
2020-02-25 12:54:45,717 : INFO : topic #3 (0.200): 0.004*"man" + 0.003*"year" + 0.003*"old" + 0.003*"state" + 0.003*"islamic" + 0.002*"killed" + 0.002*"islamic state" + 0.002*"police" + 0.002*"woman" + 0.002*"year old"
2020-02-25 12:54:45,719 : INFO : topic #4 (0.200): 0.030*"new" + 0.020*"york" + 0.020*"new york" + 0.019*"times" + 0.018*"york times" + 0.008*"trump" + 0.002*"police" + 0.002*"ban" + 0.001*"obama" + 0.001*"years"
2020-02-25 12:54:45,722 : INFO : topic diff=0.161247, rho=0.186496
2020-02-25 12:54:45,737 : INFO : PROGRESS: pa

2020-02-25 12:54:49,351 : INFO : topic #1 (0.200): 0.036*"trump" + 0.024*"breitbart" + 0.009*"clinton" + 0.008*"donald" + 0.008*"donald trump" + 0.006*"hillary" + 0.005*"obama" + 0.003*"hillary clinton" + 0.003*"white" + 0.003*"says"
2020-02-25 12:54:49,355 : INFO : topic #2 (0.200): 0.004*"court" + 0.003*"breitbart" + 0.003*"wall" + 0.002*"street" + 0.002*"supreme" + 0.002*"supreme court" + 0.002*"high" + 0.002*"school" + 0.002*"pay" + 0.002*"report"
2020-02-25 12:54:49,359 : INFO : topic #3 (0.200): 0.004*"man" + 0.003*"year" + 0.003*"state" + 0.003*"old" + 0.002*"islamic" + 0.002*"police" + 0.002*"killed" + 0.002*"islamic state" + 0.002*"woman" + 0.002*"year old"
2020-02-25 12:54:49,362 : INFO : topic #4 (0.200): 0.030*"new" + 0.020*"york" + 0.020*"new york" + 0.019*"times" + 0.018*"york times" + 0.007*"trump" + 0.002*"police" + 0.002*"obama" + 0.002*"years" + 0.001*"ban"
2020-02-25 12:54:49,365 : INFO : topic diff=0.165817, rho=0.186496
2020-02-25 12:54:49,380 : INFO : PROGRESS: pa

2020-02-25 12:54:52,076 : INFO : topic #2 (0.200): 0.004*"court" + 0.003*"breitbart" + 0.003*"wall" + 0.002*"street" + 0.002*"supreme" + 0.002*"supreme court" + 0.002*"high" + 0.002*"says" + 0.002*"school" + 0.002*"zika"
2020-02-25 12:54:52,080 : INFO : topic #3 (0.200): 0.004*"year" + 0.003*"man" + 0.003*"old" + 0.003*"state" + 0.003*"islamic" + 0.002*"year old" + 0.002*"police" + 0.002*"islamic state" + 0.002*"arrested" + 0.002*"killed"
2020-02-25 12:54:52,084 : INFO : topic #4 (0.200): 0.031*"new" + 0.021*"york" + 0.021*"new york" + 0.020*"times" + 0.019*"york times" + 0.008*"trump" + 0.002*"police" + 0.002*"ban" + 0.002*"years" + 0.001*"administration"
2020-02-25 12:54:52,086 : INFO : topic diff=0.168555, rho=0.186496
2020-02-25 12:54:52,102 : INFO : PROGRESS: pass 9, at document #34000/37503
2020-02-25 12:54:52,470 : INFO : merging changes from 2000 documents into a model of 37503 documents
2020-02-25 12:54:52,510 : INFO : topic #0 (0.200): 0.003*"breitbart" + 0.003*"health" + 0.0

In [16]:
lda.print_topics()

2020-02-25 12:54:53,954 : INFO : topic #0 (0.200): 0.003*"breitbart" + 0.003*"health" + 0.002*"care" + 0.002*"like" + 0.002*"work" + 0.002*"obamacare" + 0.002*"health care" + 0.002*"left" + 0.001*"town" + 0.001*"world"
2020-02-25 12:54:53,956 : INFO : topic #1 (0.200): 0.036*"trump" + 0.024*"breitbart" + 0.009*"clinton" + 0.008*"donald" + 0.008*"donald trump" + 0.006*"hillary" + 0.005*"obama" + 0.003*"hillary clinton" + 0.003*"says" + 0.003*"white"
2020-02-25 12:54:53,960 : INFO : topic #2 (0.200): 0.005*"court" + 0.003*"wall" + 0.003*"breitbart" + 0.002*"street" + 0.002*"supreme" + 0.002*"supreme court" + 0.002*"high" + 0.002*"says" + 0.002*"wall street" + 0.002*"school"
2020-02-25 12:54:53,963 : INFO : topic #3 (0.200): 0.004*"man" + 0.004*"year" + 0.003*"state" + 0.003*"old" + 0.003*"islamic" + 0.002*"islamic state" + 0.002*"police" + 0.002*"arrested" + 0.002*"killed" + 0.002*"woman"
2020-02-25 12:54:53,966 : INFO : topic #4 (0.200): 0.031*"new" + 0.021*"york" + 0.021*"new york" + 0

[(0,
  '0.003*"breitbart" + 0.003*"health" + 0.002*"care" + 0.002*"like" + 0.002*"work" + 0.002*"obamacare" + 0.002*"health care" + 0.002*"left" + 0.001*"town" + 0.001*"world"'),
 (1,
  '0.036*"trump" + 0.024*"breitbart" + 0.009*"clinton" + 0.008*"donald" + 0.008*"donald trump" + 0.006*"hillary" + 0.005*"obama" + 0.003*"hillary clinton" + 0.003*"says" + 0.003*"white"'),
 (2,
  '0.005*"court" + 0.003*"wall" + 0.003*"breitbart" + 0.002*"street" + 0.002*"supreme" + 0.002*"supreme court" + 0.002*"high" + 0.002*"says" + 0.002*"wall street" + 0.002*"school"'),
 (3,
  '0.004*"man" + 0.004*"year" + 0.003*"state" + 0.003*"old" + 0.003*"islamic" + 0.002*"islamic state" + 0.002*"police" + 0.002*"arrested" + 0.002*"killed" + 0.002*"woman"'),
 (4,
  '0.031*"new" + 0.021*"york" + 0.021*"new york" + 0.020*"times" + 0.019*"york times" + 0.008*"trump" + 0.002*"police" + 0.002*"ban" + 0.001*"years" + 0.001*"city"')]

In [17]:
lda_corpus = lda[corpus]
lda_corpus

In [18]:
lda_docs = [doc for doc in lda_corpus]

In [19]:
lda_docs[0:5]

[[(0, 0.015759107),
  (1, 0.32164884),
  (2, 0.01611854),
  (3, 0.6306551),
  (4, 0.015818438)],
 [(0, 0.014978485),
  (1, 0.3416641),
  (2, 0.014975045),
  (3, 0.014978591),
  (4, 0.61340374)],
 [(0, 0.21524626),
  (1, 0.024260676),
  (2, 0.4291297),
  (3, 0.3074916),
  (4, 0.023871852)],
 [(0, 0.016003853),
  (1, 0.9359847),
  (2, 0.01600418),
  (3, 0.01601126),
  (4, 0.015996063)],
 [(0, 0.015150591),
  (1, 0.015287153),
  (2, 0.015068044),
  (3, 0.015061597),
  (4, 0.9394326)]]

In [20]:
lda_model = LatentDirichletAllocation(n_components=5, # Number of topics
                                      learning_method='online',
                                      random_state=0,       
                                      n_jobs = -1  # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(X_train_cv1)

In [21]:
lda_output.

SyntaxError: invalid syntax (<ipython-input-21-e070f77ec24f>, line 1)

### NMF

In [ ]:
X_train_cv1.shape

In [ ]:
lsa = TruncatedSVD(5)
doc_topic = lsa.fit_transform(X_train_cv1)
lsa.explained_variance_ratio_

In [ ]:
lsa.components_.round(3)

In [ ]:
topic_word = pd.DataFrame(lsa.components_.round(3),
             index = ["component_1", "component_2", "component_3", "component_4", "component_5"],
             columns = cv1.get_feature_names())
topic_word